In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.read.csv('supermarket.csv', header=True)
df.show(1, vertical=True)

-RECORD 0------------------------------------
 Invoice ID              | 750-67-8428       
 Branch                  | A                 
 City                    | Yangon            
 Customer type           | Member            
 Gender                  | Female            
 Product line            | Health and beauty 
 Unit price              | 74.69             
 Quantity                | 7                 
 Tax 5%                  | 26.1415           
 Total                   | 548.9715          
 Date                    | 1/5/2019          
 Time                    | 13:08             
 Payment                 | Ewallet           
 cogs                    | 522.83            
 gross margin percentage | 4.761904762       
 gross income            | 26.1415           
 Rating                  | 9.1               
only showing top 1 row



In [3]:
df = df.withColumnRenamed('Product line', 'Product_line').withColumnRenamed('Customer type', 'Customer_type').withColumnRenamed('Invoice ID', 'Invoice_ID')
df.show(1)

+-----------+------+------+-------------+------+-----------------+----------+--------+-------+--------+--------+-----+-------+------+-----------------------+------------+------+
| Invoice_ID|Branch|  City|Customer_type|Gender|     Product_line|Unit price|Quantity| Tax 5%|   Total|    Date| Time|Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+------+-------------+------+-----------------+----------+--------+-------+--------+--------+-----+-------+------+-----------------------+------------+------+
|750-67-8428|     A|Yangon|       Member|Female|Health and beauty|     74.69|       7|26.1415|548.9715|1/5/2019|13:08|Ewallet|522.83|            4.761904762|     26.1415|   9.1|
+-----------+------+------+-------------+------+-----------------+----------+--------+-------+--------+--------+-----+-------+------+-----------------------+------------+------+
only showing top 1 row



In [4]:
df.createOrReplaceTempView('sales')
# ANY 
# query = '''select any(City) from sales'''
# spark.sql(query).show()

In [5]:
# approx_count_distinct
query = '''select approx_count_distinct(Branch) from sales'''
spark.sql(query).show()

+-----------------------------+
|approx_count_distinct(Branch)|
+-----------------------------+
|                            3|
+-----------------------------+



In [6]:
# approx_percentile ---> calaculates the values at a given percentile of distribution of values.
# The value found has percentile approximately equal to the given percentile

query = '''select approx_percentile(Quantity, 0.50) from sales'''
spark.sql(query).show()

+------------------------------------------------------------------------+
|approx_percentile(CAST(Quantity AS DOUBLE), CAST(0.50 AS DOUBLE), 10000)|
+------------------------------------------------------------------------+
|                                                                     5.0|
+------------------------------------------------------------------------+



In [7]:
# Avg
query =  '''select Product_line, avg(Quantity) from sales group by Product_line'''
spark.sql(query).show()

+--------------------+-----------------------------+
|        Product_line|avg(CAST(Quantity AS DOUBLE))|
+--------------------+-----------------------------+
|  Home and lifestyle|                      5.69375|
| Fashion accessories|            5.067415730337078|
|   Health and beauty|            5.618421052631579|
|Electronic access...|            5.711764705882353|
|  Food and beverages|            5.471264367816092|
|   Sports and travel|            5.542168674698795|
+--------------------+-----------------------------+



In [8]:
# bit_and, bit_or

query = '''select Product_line, bit_and(CAST(Quantity as INTEGER)), bit_or(CAST(Quantity as INTEGER)) from sales group by Product_line'''
spark.sql(query).show()

+--------------------+------------------------------+-----------------------------+
|        Product_line|bit_and(CAST(Quantity AS INT))|bit_or(CAST(Quantity AS INT))|
+--------------------+------------------------------+-----------------------------+
|  Home and lifestyle|                             0|                           15|
| Fashion accessories|                             0|                           15|
|   Health and beauty|                             0|                           15|
|Electronic access...|                             0|                           15|
|  Food and beverages|                             0|                           15|
|   Sports and travel|                             0|                           15|
+--------------------+------------------------------+-----------------------------+



In [9]:
# bool_and, bool_or 
query = '''select bool_and(Rating > 5), bool_or(Rating > 5),Branch from sales group by Branch'''
spark.sql(query).show()

+-----------------------------------+----------------------------------+------+
|bool_and((CAST(Rating AS INT) > 5))|bool_or((CAST(Rating AS INT) > 5))|Branch|
+-----------------------------------+----------------------------------+------+
|                              false|                              true|     B|
|                              false|                              true|     C|
|                              false|                              true|     A|
+-----------------------------------+----------------------------------+------+



In [10]:
# collect_list, collect_set
query = '''select collect_list(Branch), collect_list(Product_line), collect_set(Branch), collect_set(Product_line) from sales'''
spark.sql(query).show()

+--------------------+--------------------------+-------------------+-------------------------+
|collect_list(Branch)|collect_list(Product_line)|collect_set(Branch)|collect_set(Product_line)|
+--------------------+--------------------------+-------------------+-------------------------+
|[A, C, A, A, A, C...|      [Health and beaut...|          [C, B, A]|     [Fashion accessor...|
+--------------------+--------------------------+-------------------+-------------------------+



In [11]:
# corr
query = '''select corr(Quantity,Rating) from sales'''
spark.sql(query).show()

+------------------------------------------------------+
|corr(CAST(Quantity AS DOUBLE), CAST(Rating AS DOUBLE))|
+------------------------------------------------------+
|                                  -0.01581490462716...|
+------------------------------------------------------+



In [12]:
# count
query = '''select Product_line, count(Product_line) from sales group by Product_line'''
spark.sql(query).show()

+--------------------+-------------------+
|        Product_line|count(Product_line)|
+--------------------+-------------------+
|  Home and lifestyle|                160|
| Fashion accessories|                178|
|   Health and beauty|                152|
|Electronic access...|                170|
|  Food and beverages|                174|
|   Sports and travel|                166|
+--------------------+-------------------+



In [13]:
# -- count_if
query = '''select COUNT_IF(Rating>5), Branch from sales group by Branch'''
spark.sql(query).show()

+-----------------------------------+------+
|count_if((CAST(Rating AS INT) > 5))|Branch|
+-----------------------------------+------+
|                                220|     B|
|                                231|     C|
|                                234|     A|
+-----------------------------------+------+



In [14]:
# covar_pop ---> coverience of a population
# covar_samp ---> Returns the sample covariece 
query='''select covar_pop(Quantity, Rating) as covar_pop, covar_samp(Quantity, Rating) as covar_samp from sales'''
spark.sql(query).show()

+-------------------+--------------------+
|          covar_pop|          covar_samp|
+-------------------+--------------------+
|-0.0793770000000004|-0.07945645645645687|
+-------------------+--------------------+



In [15]:
# first, last, first_value, last_value
query = '''select first(Invoice_ID), last(Invoice_ID), first_value(Invoice_ID), last_value(Invoice_ID) from sales'''
spark.sql(query).show()

+-----------------+----------------+-----------------------+----------------------+
|first(Invoice_ID)|last(Invoice_ID)|first_value(Invoice_ID)|last_value(Invoice_ID)|
+-----------------+----------------+-----------------------+----------------------+
|      750-67-8428|     849-09-3807|            750-67-8428|           849-09-3807|
+-----------------+----------------+-----------------------+----------------------+



In [16]:
# kurtosis, max, min, mean
query = '''select kurtosis(Quantity), max(Quantity), min(Quantity), mean(Quantity) from sales'''
spark.sql(query).show()

+----------------------------------+-------------+-------------+------------------------------+
|kurtosis(CAST(Quantity AS DOUBLE))|max(Quantity)|min(Quantity)|mean(CAST(Quantity AS DOUBLE))|
+----------------------------------+-------------+-------------+------------------------------+
|               -1.2154719990982275|            9|            1|                          5.51|
+----------------------------------+-------------+-------------+------------------------------+



In [17]:
# max_by ---> Returns the value of x associated with maximum value of y
# min_by ---> Returns the value x associated with the minimum value of y
query = '''select max_by(Rating, Quantity), min_by(Rating, Quantity) from sales'''
spark.sql(query).show()

+------------------------+------------------------+
|max_by(Rating, Quantity)|min_by(Rating, Quantity)|
+------------------------+------------------------+
|                     7.4|                     4.1|
+------------------------+------------------------+



In [18]:
# percentile
query = '''select percentile(Quantity, 0.3) as per, percentile(Rating, array(0.3,0.4)) as per_arr, percentile_approx(Quantity, array(0.3,0.35,0.4,0.5), 100) as per_approx from sales'''
spark.sql(query).show()

+---+----------+--------------------+
|per|   per_arr|          per_approx|
+---+----------+--------------------+
|4.0|[5.9, 6.4]|[3.0, 4.0, 4.0, 5.0]|
+---+----------+--------------------+



In [19]:
# skewness, std, stddev, stddev_pop, stddev_sam
query = '''select skewness(Quantity), std(Quantity), stddev(Quantity), stddev_pop(Quantity), stddev_samp(Quantity) from sales'''
spark.sql(query).show()

+----------------------------------+-----------------------------+--------------------------------+------------------------------------+-------------------------------------+
|skewness(CAST(Quantity AS DOUBLE))|std(CAST(Quantity AS DOUBLE))|stddev(CAST(Quantity AS DOUBLE))|stddev_pop(CAST(Quantity AS DOUBLE))|stddev_samp(CAST(Quantity AS DOUBLE))|
+----------------------------------+-----------------------------+--------------------------------+------------------------------------+-------------------------------------+
|              0.012921628351325263|           2.9234305954556956|              2.9234305954556956|                  2.9219685145463146|                   2.9234305954556956|
+----------------------------------+-----------------------------+--------------------------------+------------------------------------+-------------------------------------+



# Window functions

In [20]:
query = '''select Product_line, Quantity,  percent_rank() over (partition by Product_line order by Quantity) as per_rank from sales'''
spark.sql(query).show()

+------------------+--------+-------------------+
|      Product_line|Quantity|           per_rank|
+------------------+--------+-------------------+
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|       1|                0.0|
|Home and lifestyle|      10|0.08176100628930817|
|Home and lifestyle|      10|0.08176100628930817|
|Home and lifestyle|      10|0.08176100628930817|
|Home and lifestyle|      10|0.08176100628930817|


In [21]:
query = '''select Product_line, Quantity,  row_number() over (partition by Product_line order by Quantity) as row_num from sales'''
spark.sql(query).show()

+------------------+--------+-------+
|      Product_line|Quantity|row_num|
+------------------+--------+-------+
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      2|
|Home and lifestyle|       1|      3|
|Home and lifestyle|       1|      4|
|Home and lifestyle|       1|      5|
|Home and lifestyle|       1|      6|
|Home and lifestyle|       1|      7|
|Home and lifestyle|       1|      8|
|Home and lifestyle|       1|      9|
|Home and lifestyle|       1|     10|
|Home and lifestyle|       1|     11|
|Home and lifestyle|       1|     12|
|Home and lifestyle|       1|     13|
|Home and lifestyle|      10|     14|
|Home and lifestyle|      10|     15|
|Home and lifestyle|      10|     16|
|Home and lifestyle|      10|     17|
|Home and lifestyle|      10|     18|
|Home and lifestyle|      10|     19|
|Home and lifestyle|      10|     20|
+------------------+--------+-------+
only showing top 20 rows



In [23]:
query = '''select Product_line, Quantity,  rank() over (partition by Product_line order by Quantity) as row_num from sales'''
spark.sql(query).show()

+------------------+--------+-------+
|      Product_line|Quantity|row_num|
+------------------+--------+-------+
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|       1|      1|
|Home and lifestyle|      10|     14|
|Home and lifestyle|      10|     14|
|Home and lifestyle|      10|     14|
|Home and lifestyle|      10|     14|
|Home and lifestyle|      10|     14|
|Home and lifestyle|      10|     14|
|Home and lifestyle|      10|     14|
+------------------+--------+-------+
only showing top 20 rows



In [25]:
query = '''select Product_line, Quantity,  cume_dist() over (partition by Product_line order by Quantity) as cum_dist from sales'''
spark.sql(query).show()

+------------------+--------+--------+
|      Product_line|Quantity|cum_dist|
+------------------+--------+--------+
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|       1| 0.08125|
|Home and lifestyle|      10|  0.1875|
|Home and lifestyle|      10|  0.1875|
|Home and lifestyle|      10|  0.1875|
|Home and lifestyle|      10|  0.1875|
|Home and lifestyle|      10|  0.1875|
|Home and lifestyle|      10|  0.1875|
|Home and lifestyle|      10|  0.1875|
+------------------+--------+--------+
only showing top 20 rows

